# Dividend payout trend

In [1]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import pandas as pd
import datetime as dt
import calendar as cal
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "end_to_end_project"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", module="scipy", message="^internal gelsd")

from IPython.display import display
pd.options.display.max_columns = 50

## Read the high yields stock and dividends file

In [12]:
df_stocks =  pd.read_csv('../dataset/KLSE_category_2.csv')
display(df_stocks)
df_dividends = pd.read_csv('../dataset/KLSE_dividends_i3investor.csv')
display(df_dividends)

,code,name,yield_sum,yield_mean
0,5191,Tambun Indah Land Bhd (TAMB),49.69,16.56330
1,5272,Ranhill Holdings Ltd (RANH),48.11,24.05500
2,5161,JCY International Bhd (JCYI),47.34,23.67000
3,5254,Boustead Plantations Bhd (BOPL),47.29,23.64500
4,6084,Star Media Group Bhd (STAR),45.65,15.21670
5,5078,Marine & General Bhd (MARI),45.45,45.45000
6,8524,Taliworks Corporation Bhd (TWRK),42.03,21.01500
7,4502,Media Prima Bhd (MPRM),41.03,20.51500
8,5010,Tong Herr Resources Bhd (THRB),40.67,13.55670
9,3859,Magnum Bhd (MAGM),39.69,19.84500


,code,ann_date,ex_date,payment_date,dividend_type,subject,amount
0,0012,06-Nov-2017,24-Nov-2017,12-Dec-2017,DIVIDEND,First Interim Dividend,RM 0.018
1,0012,11-May-2017,23-May-2017,25-May-2017,BONUS_ISSUE,Bonus Issue,1 : 4
2,0012,15-Nov-2016,25-Nov-2016,13-Dec-2016,DIVIDEND,Interim Dividend,RM 0.018
3,0012,24-Nov-2015,16-Dec-2015,08-Jan-2016,DIVIDEND,Interim Dividend,7.0000%
4,0012,28-Nov-2014,18-Dec-2014,05-Jan-2015,DIVIDEND,Interim Dividend,7%
5,0012,29-Nov-2013,18-Dec-2013,08-Jan-2014,DIVIDEND,Interim Dividend,6%
6,0012,27-Nov-2012,21-Dec-2012,11-Jan-2013,DIVIDEND,Interim Dividend,6%
7,0012,22-Nov-2011,19-Dec-2011,10-Jan-2012,DIVIDEND,Interim Dividend,6%
8,0012,29-Nov-2010,21-Dec-2010,11-Jan-2011,DIVIDEND,Interim Dividend,6%
9,0012,17-Dec-2009,31-Dec-2009,19-Jan-2010,DIVIDEND,Interim Dividend,8%


## Read the trending and due ex-dividend dates for next 2 months

In [8]:
for stock in df_stocks:
    print(stock)

code
name
yield_sum
yield_mean
